<a href="https://colab.research.google.com/github/Peppecoding/Olist-Brazilian-e-commerce-analisys/blob/main/Stat_mini_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Project Objective
In this project, we’ll analyze the Olist Brazilian e-commerce dataset to extract meaningful business insights. We'll explore customer behavior, shipping times, payment methods, and satisfaction ratings. Then, we'll validate hypotheses using A/B testing to understand what impacts customer experience.

## 2. Import Libraries and Load Datasets

In [29]:
import scipy.stats as stats
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

df_customers = pd.read_csv('olist_customers_dataset.csv')
df_order_items = pd.read_csv('olist_order_items_dataset.csv')
df_payments = pd.read_csv('olist_order_payments_dataset.csv')
df_reviews = pd.read_csv('olist_order_reviews_dataset.csv')
df_orders = pd.read_csv('olist_orders_dataset.csv')
df_products = pd.read_csv('olist_products_dataset.csv')
df_sellers = pd.read_csv('olist_sellers_dataset.csv')


In [30]:
print(df_orders.info())
print(df_order_items.info())
print(df_payments.info())
print(df_reviews.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  ob

 ## 3. Initial Exploration

In [31]:
# Información general sobre el dataset de órdenes
df_orders.info()  # Nos dice qué columnas hay, tipos de datos, y cuántos valores nulos existen.
df_orders.head()  # Muestra las primeras filas para tener una idea visual.

# Podemos hacer lo mismo para otros datasets, por ejemplo:
df_order_items.info()
df_payments.info()
df_reviews.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 

In [32]:
# Convertir columnas de fecha a tipo datetime en el dataframe de órdenes
df_orders['order_purchase_timestamp'] = pd.to_datetime(df_orders['order_purchase_timestamp'])
df_orders['order_delivered_customer_date'] = pd.to_datetime(df_orders['order_delivered_customer_date'])
df_orders['order_estimated_delivery_date'] = pd.to_datetime(df_orders['order_estimated_delivery_date'])

# Comprobamos si existen valores nulos en el dataframe de órdenes y de reviews
print(df_orders.isnull().sum())
print(df_reviews.isnull().sum())


order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64
review_id                      0
order_id                       0
review_score                   0
review_comment_title       87656
review_comment_message     58247
review_creation_date           0
review_answer_timestamp        0
dtype: int64


## 4. Data Cleaning

In [33]:
df_orders['order_purchase_timestamp'] = pd.to_datetime(df_orders['order_purchase_timestamp'])
df_orders['order_delivered_customer_date'] = pd.to_datetime(df_orders['order_delivered_customer_date'])
df_orders['order_estimated_delivery_date'] = pd.to_datetime(df_orders['order_estimated_delivery_date'])

df_reviews['review_creation_date'] = pd.to_datetime(df_reviews['review_creation_date'])
df_reviews['review_answer_timestamp'] = pd.to_datetime(df_reviews['review_answer_timestamp'])

df_order_items['shipping_limit_date'] = pd.to_datetime(df_order_items['shipping_limit_date'])


In [40]:
df_reviews = df_reviews[['order_id', 'review_score']]

## 5. Descriptive Statistics

### Total Order Price

In [35]:
# Calculate total price per order
order_prices = df_order_items.groupby('order_id')['price'].sum().reset_index()
order_prices.columns = ['order_id', 'total_price']

# Merge with orders
df_order_prices = df_orders.merge(order_prices, on='order_id', how='left')

# Summary statistics
df_order_prices['total_price'].describe()


,total_price
count,98666.000000
mean,137.754076
std,210.645145
min,0.850000
25%,45.900000
50%,86.900000
75%,149.900000
max,13440.000000


In [36]:
# Histogram
px.histogram(df_order_prices, x='total_price', nbins=50, title='Order Price Distribution')


### Top Product Categories

In [37]:
# Merge products with order items
products_by_order = df_order_items.merge(df_products, on='product_id', how='left')

# Count by category
category_counts = products_by_order['product_category_name'].value_counts().reset_index()
category_counts.columns = ['product_category_name', 'order_count']

# Bar chart
px.bar(category_counts.head(10), x='product_category_name', y='order_count',
       title='Top 10 Product Categories by Number of Orders')


### Payment Method Distribution

In [38]:
payment_counts = df_payments['payment_type'].value_counts().reset_index()
payment_counts.columns = ['payment_type', 'count']

px.pie(payment_counts, values='count', names='payment_type',
       title='Payment Method Distribution')


In [42]:
# Boxplot
df_order_prices['shipping_time'] = (
    df_order_prices['order_delivered_customer_date'] - df_order_prices['order_purchase_timestamp']
).dt.days
df_order_prices[['shipping_time']].head()



,shipping_time
0,8.0
1,13.0
2,9.0
3,13.0
4,2.0


In [43]:
px.box(df_order_prices, y='shipping_time', title='Shipping Time (days)')

 ## 7. Customer Satisfaction Overview

In [ ]:
# Merge reviews with orders
df_full = df_order_prices.merge(df_reviews, on='order_id', how='left')

# Satisfied if review_score >= 4
df_full['is_satisfied'] = df_full['review_score'] >= 4
satisfaction_rate = df_full['is_satisfied'].mean()
print(f'Satisfaction rate: {round(satisfaction_rate * 100, 2)}%')


## 8. Hypothesis Testing – Do Late Deliveries Lower Satisfaction?

### Hypothesis

H0 (null): There is no difference in satisfaction between on-time and late deliveries.

H1 (alt): Late deliveries result in lower satisfaction.

In [ ]:
# Identify late deliveries
df_full['delivered_late'] = df_full['order_delivered_customer_date'] > df_full['order_estimated_delivery_date']

# Split into groups
on_time = df_full[df_full['delivered_late'] == False]['review_score']
late = df_full[df_full['delivered_late'] == True]['review_score']

# Perform t-test
t_stat, p_value = stats.ttest_ind(on_time.dropna(), late.dropna())
print(f"T-statistic: {t_stat}, p-value: {p_value}")


### Interpretation:
p < 0.05 → Reject H0 → Significant difference → Late deliveries impact satisfaction.

p ≥ 0.05 → Fail to reject H0 → No evidence of difference.


##  9. Visualizing A/B Test Results

In [ ]:
avg_scores = df_full.groupby('delivered_late')['review_score'].mean().reset_index()
avg_scores['delivered_late'] = avg_scores['delivered_late'].map({True: 'Late', False: 'On Time'})

px.bar(avg_scores, x='delivered_late', y='review_score',
       title='Average Satisfaction Score: On-Time vs. Late Deliveries')

## 10. Conclusion
Customers tend to buy products mostly from categories like “bed_bath_table” and “health_beauty”.

Credit cards are the dominant payment method.

The average shipping time is X days.

The customer satisfaction rate is Y%.

A/B testing shows that late deliveries (do/do not) significantly reduce satisfaction.

##  Next Steps
Include more hypotheses (e.g., payment method vs. completion rate, product category vs. order value).

Explore seasonal patterns in shipping delays or order volume.

Build a dashboard to summarize KPIs for Olist’s management.

